# The MOOC Pivot: Base code

Author: José A. Rupérez Valiente
Email: jruipere@mit.edu

## Libraries

In [1]:
import pandas as pd
import numpy as np

## Data loading

In [2]:
# Importing the person_course dataset
all_person_course = pd.read_csv('person_course.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Importing course metadata
course_metadata = pd.read_csv('course_metadata.csv')

In [4]:
# Importing country metadata from the United Nations
country_metadata = pd.read_csv('country_metadata.csv', keep_default_na=False, na_values = [''])

## Data cleaning and filtering

In [5]:
all_person_course.rename(columns = {'cc_by_ip':'alpha-2'}, inplace = True)

In [6]:
# keeping only students, removing course staff
all_person_course = all_person_course[all_person_course['roles'] == 'Student']

In [7]:
# Consolidating nationality of each user -- This is done to fix some missing values 
all_person_course = pd.merge(all_person_course[['course_id', 'user_id', 'completed', 'roles', 'viewed',
       'explored', 'certified', 'mode', 'prs_reason_lc', 'prs_intent_verified',
       'prs_intent_lecture', 'prs_intent_assess', 'prs_intent']],
       all_person_course.groupby(by = 'user_id', as_index=False).agg({'alpha-2':'first'}),
       on = 'user_id', how='left')

## Data merging

In [8]:
# merging both the course and country metadata into the main person_course file
all_person_course = pd.merge(all_person_course, course_metadata, on = 'course_id', how = 'left')

In [9]:
all_person_course = pd.merge(all_person_course, country_metadata, on = 'alpha-2', how = 'left')

## Analysis

### Table S2 - HarvardX and MITx Number of Enrolments and Certifications by Year

In [10]:
# grouping by year and HDI category and summing up enrolments and certifications
enrolments_certifications_per_year_hdi = all_person_course.groupby(['year', 
                                         'human_development_category']).agg({'user_id': 'count',
                                         'certified':'sum'}).reset_index().rename(columns = {'user_id':'n_enrolments',
                                                                                             'certified':'n_certified'})

In [11]:
enrolments_certifications_per_year_hdi['human_development_category'] = pd.Categorical(enrolments_certifications_per_year_hdi['human_development_category'],
                                                                                     categories=["very_high_human_development","high_human_development","medium_human_development", "low_human_development"],
                                                                                     ordered = True)

In [12]:
enrolments_certifications_per_year_hdi.sort_values(['year', 'human_development_category'], inplace=True)

In [13]:
# adding totals by cohort
enrolments_certifications_per_year_hdi = pd.merge(enrolments_certifications_per_year_hdi, 
        enrolments_certifications_per_year_hdi.groupby('year').agg({'n_enrolments': 'sum',
                                                                                  'n_certified': 'sum'}).reset_index().rename(columns = {'n_enrolments' : 'total_enrolments_year',
                                                                                                                                         'n_certified': 'total_certified_year'}),
        on = 'year', how = 'left')

In [14]:
enrolments_certifications_per_year_hdi['p_enrolments_within_year'] = round(100*enrolments_certifications_per_year_hdi['n_enrolments']/enrolments_certifications_per_year_hdi['total_enrolments_year'],2)
enrolments_certifications_per_year_hdi['p_certified_within_year'] = round(100*enrolments_certifications_per_year_hdi['n_certified']/enrolments_certifications_per_year_hdi['total_certified_year'],2)

In [15]:
enrolments_certifications_per_year_hdi.drop(columns = ['total_enrolments_year', 'total_certified_year'], inplace=True)

In [16]:
enrolments_certifications_per_year_hdi

,year,human_development_category,n_enrolments,n_certified,p_enrolments_within_year,p_certified_within_year
0,2012-13,very_high_human_development,205955,10798,59.28,61.98
1,2012-13,high_human_development,54483,2476,15.68,14.21
2,2012-13,medium_human_development,74395,3351,21.41,19.24
3,2012-13,low_human_development,12616,796,3.63,4.57
4,2013-14,very_high_human_development,1359191,43235,65.55,71.55
5,2013-14,high_human_development,318683,8429,15.37,13.95
6,2013-14,medium_human_development,357971,7685,17.26,12.72
7,2013-14,low_human_development,37780,1075,1.82,1.78
8,2014-15,very_high_human_development,1583635,55674,62.82,71.20
9,2014-15,high_human_development,414912,10903,16.46,13.94


In [17]:
enrolments_certifications_per_year_hdi.to_csv('Table_S2.csv', index = False)

### Table S4 - Mean, Median and Std. Deviation of Participation/Completion by HarvardX and MITx Course

In [18]:
# aggregating participation and completion by course
data_by_course = all_person_course.groupby('course_id').agg({'user_id': 'count',
                                           'viewed': 'sum', 'explored': 'sum',
                                           'completed': 'sum',
                                           'certified': 'sum'}).reset_index().rename(columns = {'user_id':'registrations'})

In [19]:
# computing percentage from registrants
data_by_course['p_participants'] = round(100*data_by_course['viewed']/data_by_course['registrations'],2)

In [20]:
# computing percentages from participants
data_by_course[['p_explored','p_completed','p_certified']] = round(100*data_by_course[['explored','completed','certified']].div(data_by_course.viewed, axis=0),2)

In [21]:
# Removing some infinite rows due to some data issues
data_by_course = data_by_course[~(data_by_course['p_completed'] == np.inf)]

In [22]:
# Computing mean, median and std. deviation by course
agg_data_by_course = round(data_by_course.agg({'p_participants':['mean', 'median', 'std'],
                                               'p_explored':['mean', 'median', 'std'],
                                               'p_completed':['mean', 'median', 'std'],
                                               'p_certified':['mean', 'median', 'std']}),2)

In [23]:
agg_data_by_course

,p_participants,p_explored,p_completed,p_certified
mean,48.02,25.59,6.39,5.03
median,51.01,19.84,4.42,2.88
std,15.49,19.54,6.62,6.04


In [24]:
agg_data_by_course.to_csv('Table_S4.csv', index = False)

### Table S1 - HarvardX and MITx Churn Rate

In [25]:
year_to_num = {'2012-13':1, '2013-14':2, '2014-15':3, '2015-16':4,'2016-17':5, '2017-18':6}
num_to_year = {1:'2012-13', 2:'2013-14', 3:'2014-15', 4:'2015-16', 5:'2016-17', 6:'2017-18'}

In [26]:
# keeping only participants
all_person_course = all_person_course[all_person_course['viewed'] == 1]

In [27]:
all_person_course.set_index('user_id', inplace=True)

In [28]:
# need to turn into a numeric to do the min afterwards
all_person_course['year2'] = all_person_course['year'].map(year_to_num)

In [29]:
# we compute the registration year per user
all_person_course['registration_year'] = all_person_course.groupby(level=0)['year2'].min()

In [30]:
# group by registration year and runnig year to compute number of unique users per each one
attrition_by_year = all_person_course.reset_index().groupby(['registration_year', 'year2']).agg({'user_id': pd.Series.nunique}).reset_index()

In [31]:
attrition_by_year = attrition_by_year.rename(columns={'user_id': 'unique_learners', 'year2': 'year'})

In [32]:
# add the initial size of each registration year cohort
attrition_by_year = pd.merge(attrition_by_year, 
                             attrition_by_year.groupby('registration_year').agg({'unique_learners':'first'}).reset_index().rename(columns={'unique_learners':'cohort_learners'}),
                             on = 'registration_year', how = 'left')

In [33]:
# computing retention in each running year per cohort
attrition_by_year['percentage_retention'] = round(100*attrition_by_year['unique_learners']/attrition_by_year['cohort_learners'],2)

In [34]:
attrition_by_year.drop(columns = 'cohort_learners', inplace=True)

In [35]:
attrition_by_year['registration_year'] = attrition_by_year['registration_year'].map(num_to_year)
attrition_by_year['year'] = attrition_by_year['year'].map(num_to_year)

In [36]:
attrition_by_year

,registration_year,year,unique_learners,percentage_retention
0,2012-13,2012-13,192255,100.00
1,2012-13,2013-14,72093,37.50
2,2012-13,2014-15,46494,24.18
3,2012-13,2015-16,36089,18.77
4,2012-13,2016-17,21690,11.28
5,2012-13,2017-18,15059,7.83
6,2013-14,2013-14,755800,100.00
7,2013-14,2014-15,205814,27.23
8,2013-14,2015-16,146713,19.41
9,2013-14,2016-17,76780,10.16


In [37]:
attrition_by_year.to_csv('Table_S1.csv', index = False)

### Table S3 HarvardX and MITx Course Completion by Year and Cohort of Learners

In [38]:
# this is the cohort of participants (we already filtered to keep only participants before, so no need to do it again here)
completion_participants = all_person_course[['year', 'viewed', 'completed']].groupby('year').sum().reset_index()

In [39]:
completion_participants['p_completed'] = round(100*(completion_participants['completed']/completion_participants['viewed']), 2)
completion_participants['cohort'] = 'participants'

In [40]:
# subsetting registrations of students where they self-reported intention to complete
# due to differences in survey fields over the years we need to use several fields
completion_intended = all_person_course[(all_person_course['prs_reason_lc'] == 3) 
                  | (all_person_course['prs_intent'] == 3)
                  | (all_person_course['prs_intent_verified'] == 1) 
                  | (all_person_course['prs_intent_assess'] == 3)
                  | (all_person_course['prs_intent_lecture'] == 3)][['year', 'viewed', 'completed']].groupby('year').sum().reset_index()

In [41]:
completion_intended['p_completed'] = round(100*(completion_intended['completed']/completion_intended['viewed']), 2)
completion_intended['cohort'] = 'intend_complete'

In [42]:
# subsetting registrations of students that paid to enter the verified track
completition_verified = all_person_course[all_person_course['mode'] == 'verified'][['year', 'viewed', 'completed']].groupby('year').sum().reset_index()

In [43]:
completition_verified['p_completed'] = round(100*(completition_verified['completed']/completition_verified['viewed']), 2)
completition_verified['cohort'] = 'verified'

In [44]:
# concatenating the three cohorts together
completion_by_cohort = pd.concat([completion_participants, completion_intended, completition_verified]).rename(columns = {'viewed':'n_participants',
                                                                                                                          'completed':'n_completed'})

In [45]:
# removing Year 2012-13 since it did not have surveys or verified track
completion_by_cohort = completion_by_cohort[~(completion_by_cohort['year'] == '2012-13')]

In [46]:
completion_by_cohort

,year,n_participants,n_completed,p_completed,cohort
1,2013-14,1219030,60285.0,4.95,participants
2,2014-15,1333780,78085.0,5.85,participants
3,2015-16,1958222,80495.0,4.11,participants
4,2016-17,1097759,43726.0,3.98,participants
5,2017-18,980589,30650.0,3.13,participants
0,2013-14,178909,20593.0,11.51,intend_complete
1,2014-15,191578,33986.0,17.74,intend_complete
2,2015-16,209333,34416.0,16.44,intend_complete
3,2016-17,74116,11952.0,16.13,intend_complete
4,2017-18,42005,6478.0,15.42,intend_complete


In [47]:
completion_by_cohort.to_csv('Table_S3.csv', index = False)